This notebook provides an overview of additional customizing functions that can be used in scripting.

## Callbacks

Register a function to be called at $t=0$ and after each timestep using <code style="color:#c586c0">problem.add_callback(function)</code>. 

The callback takes no arguments, required data can be accessed using the `problem` instance via closure.

In [ ]:
from GaPFlow import Problem
problem = Problem()

# Storage for tracked quantities
history = {'step': [], 'mass': [], 'v_max': []}

def track_quantities():
    history['step'].append(problem.step)
    history['mass'].append(problem.mass)
    history['v_max'].append(problem.v_max)

problem.add_callback(track_quantities)

## Customized Boundary Conditions

Register custom boundary condition functions using <code style="color:#c586c0">problem.set_bc_function(var, boundary, bc_function)</code>.
- `var`: `rho`, `jx`, `jy`
- `boundary`: `W`, `E`, `S`, or `N`
- `bc_function`: customized bc function, see below

The <code style="color:#c586c0">bc_function</code> receives a `BCContext` object with the following attributes: 
- `problem`: The GaPFlow Problem instance.
- `required_shape`: Required shape of the returned array.
- `ghost_slice`: Slice for accessing ghost cells.
- `interior_slice`: Slice for accessing adjacent interior cells.
- `y_norm`: Normalized y-coordinates
- `x_norm`: Normalized x-coordinates

Note that the boundary might be split between several processes in parallel execution. In this case it important that you either
- use `required_shape` e.g. for creating an correspondingly-shaped array that you fill uniformly
- determine bc array using operations on `problem.q` fields using `[ghost_slice]` or `[interior_slice]` slices
- `x_norm`, `y_norm` for consistently imposing location-dependent boundary values
- or a combination of these.


For example, if we want a uniform, time-dependent density boundary condition at `W`:

In [ ]:
from GaPFlow.parallel import BCContext
import numpy as np

def time_dependent_density_bc(ctx: BCContext):
    """Time-varying density at west boundary."""
    t = ctx.problem.simtime
    rho_new = 1 + 0.1 * np.sin(t)
    rho_bc = np.ones(ctx.required_shape) * rho_new
    return rho_bc

problem.set_bc_function('rho', 'W', time_dependent_density_bc)

Or if we want to impose a Couette-Flow velocity profile for the mass flux $j_x$ at `W`:

We need:
- `y_norm` for correct location dependency of $u$ in parallel runs
- read $\rho$ from solution field to calculate $j_x = \rho \cdot u$

In [ ]:
max_u = 10.  # m/s Couette flow from 0-10 m/s along y-direction

def couette_flow_bc(ctx: BCContext):
    """Couette flow velocity profile at west boundary."""
    u_bc = ctx.y_norm * max_u  # using normalized y-coordinates for velocity profile
    rho_bc = ctx.problem.q[0, ctx.ghost_slice]  # get density from ghost cells
    jx_bc = rho_bc * u_bc
    return jx_bc

problem.set_bc_function('jx', 'W', couette_flow_bc)

## Changing Topography

Modify the gap height before and/or during simulation. There are two possibilities:

---

**You want to apply a function that maps $x, y, t, ... \rightarrow h$**

To be consistent between serial and parallel runs:

- use `problem.topo.local_shape` for creating arrays that you want to fill uniformly
- `problem.topo.xx`, `problem.topo.yy` for dimensionalized location dependency - already in the required shape
- `problem.topo.xx_norm`, `problem.topo.yy_norm` for normalized $[0,1 ]$ location dependency - already in the required shape
- `problem.topo.h` the current height field - already in the required shape
- use the function <code style="color:#569cd6">problem.topo.set_mapped_height(h_arr)</code>

---

**You have a height profile e.g. from a dataset that you want to apply**

- the array must have shape `problem.topo.global_shape`
- use the function <code style="color:#569cd6">problem.topo.set_global_height(h_arr)</code>
- if running in parallel, this height field will be correctly distributed onto the processes

---

Setting a customized height profile from a dataset after problem initialization:

In [ ]:
from GaPFlow import Problem
import numpy as np

def load_height_field(filename: str) -> np.ndarray:
      """Load height array from file (user implementation)."""
      return np.loadtxt(filename)

# Initialize problem
problem = Problem()

# Set customized topography
h_custom = load_height_field('topo_roughness.txt')
assert(h_custom.shape == problem.topo.global_shape)
problem.topo.set_global_height(h_custom)

# Run problem
problem.run()

Implementing a time- and location-dependent height change by registering a callback function:

In [ ]:
from GaPFlow import Problem
import numpy as np

problem = Problem()

h_base = np.ones(problem.topo.local_shape)

def time_dependent_height():
    """x-dependent height change oscillating with time."""
    t = problem.simtime
    dh = 0.1 * problem.topo.xx_norm * np.sin(t)  # example time variation with x-dependency
    h_new = h_base + dh
    problem.topo.set_mapped_height(h_new)

problem.add_callback(time_dependent_height)

problem.run()

## Scripting in Parallel Runs

**Launching parallel runs:**

```bash
mpirun -n 4 python my_script.py
```

**Domain decomposition:** The grid is split across processes. Each process only sees its local subdomain. Access decomposition info via `problem.decomp`:

- `problem.decomp.rank` - MPI rank of this process
- `problem.decomp.size` - total number of processes
- `problem.decomp.is_at_xW`, `is_at_xE`, `is_at_yS`, `is_at_yN` - boundary ownership
- `problem.topo.xx`, `problem.topo.yy` - local coordinates (use for location-dependent operations)

**Gathering global fields:** Use <code style="color:#c586c0">problem.decomp.gather_global(field)</code> to collect local fields to rank 0 for post-processing or plotting:

In [ ]:
if problem.decomp.rank == 0:
    rho_global = problem.decomp.gather_global(problem.q[0])
    # plot or analyze rho_global as needed